<a href="https://colab.research.google.com/github/Ilurusheshasai/Ilurusheshasai/blob/main/757.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!find / -name "train.zip" -type f

find: ‘/proc/52/task/52/net’: Invalid argument
find: ‘/proc/52/net’: Invalid argument
find: ‘/proc/578/task/578/net’: Invalid argument
find: ‘/proc/578/net’: Invalid argument
/content/drive/MyDrive/train.zip
^C


In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
import tensorflow as tf

# Create train, validation, and test datasets from a zip file
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/train',
    batch_size=32,
    validation_split=0.3,  # Use 70% of the data for training
    subset='training',
    seed=123
)


Found 13841 files belonging to 200 classes.
Using 9689 files for training.


In [ ]:

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/train',
    batch_size=32,
    validation_split=0.3,  # Use 20% of the data for validation
    subset='validation',
    seed=123
)



Found 13841 files belonging to 200 classes.
Using 4152 files for validation.


In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/train',
    batch_size=32,
    validation_split=0.3,  # Use 10% of the data for testing
    subset='validation',
    seed=456
)



Found 13841 files belonging to 200 classes.
Using 4152 files for validation.


In [46]:
from tensorflow.keras import layers, models, optimizers, regularizers
# Create a CNN model
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200,activation = 'softmax')
])



In [47]:
from tensorflow.keras.callbacks import EarlyStopping
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_accuracy', patience=5)

In [48]:

# Train the model
history = model.fit(train_dataset, epochs=25, validation_data=val_dataset, callbacks=[early_stop])


Epoch 1/25
303/303 [==============================] - 38s 111ms/step - loss: 4.5109 - accuracy: 0.0321 - val_loss: 4.1856 - val_accuracy: 0.0617
Epoch 2/25
303/303 [==============================] - 33s 108ms/step - loss: 4.1859 - accuracy: 0.0575 - val_loss: 4.0729 - val_accuracy: 0.0862
Epoch 3/25
303/303 [==============================] - 34s 109ms/step - loss: 4.0818 - accuracy: 0.0745 - val_loss: 3.9797 - val_accuracy: 0.0942
Epoch 4/25
303/303 [==============================] - 34s 112ms/step - loss: 4.0018 - accuracy: 0.0910 - val_loss: 3.9768 - val_accuracy: 0.1089
Epoch 5/25
303/303 [==============================] - 33s 108ms/step - loss: 3.9505 - accuracy: 0.0997 - val_loss: 3.9348 - val_accuracy: 0.1171
Epoch 6/25
303/303 [==============================] - 33s 108ms/step - loss: 3.9072 - accuracy: 0.1170 - val_loss: 3.9953 - val_accuracy: 0.1166
Epoch 7/25
303/303 [==============================] - 33s 108ms/step - loss: 3.8576 - accuracy: 0.1318 - val_loss: 3.9773 - val_ac

In [49]:

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {accuracy:.2f}')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy:.2f}')

130/130 [==============================] - 9s 63ms/step - loss: 5.9595 - accuracy: 0.0987
Validation accuracy: 0.10
130/130 [==============================] - 10s 73ms/step - loss: 3.1828 - accuracy: 0.5267
Test accuracy: 0.53
